In [1]:
import os

os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [2]:
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
import glob
from IPython.core.display_functions import clear_output

In [ ]:
import sys  
sys.path.insert(1, '../../../')


import generate_dataset
import experiment, autoencoders

### Parameters

In [4]:
num_endmembers = 5
spectral_size = 1000
image_size = 100
epochs = 10
ae_loss = autoencoders.SAD
model_replicates = 5

scenes = ['chessboard', 'gaussian', 'dirichlet']

data_seeds = [1, 2, 3, 4, 5]
experiment_seed = 42

data_folder = 'generated data'
results_folder = 'results'

In [5]:
def display_results(metrics):
    results_order = ['PCA', 'N-FINDR + FCLS', 'VCA + FCLS', 'Dense AE', 'Convolutional AE', 'Transformer AE', 'Convolutional Transformer AE']

    metrics['SAD'] = metrics[['SAD_mean', 'SAD_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)
    metrics['PCC'] = metrics[['PCC_mean', 'PCC_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)
    metrics['MSE'] = metrics[['MSE_mean', 'MSE_std']].apply(lambda x: ' ± '.join(x.round(3).astype(str)), axis=1)

    display(metrics.set_index('index').reindex(results_order)[['SAD', 'PCC', 'MSE']])

In [6]:
results = {}

### Ideal scenario

In [7]:
folder = 'ideal'

results[folder] = {}
for scene in scenes:
    # Create folders
    scene_folder = os.path.join(data_folder, folder, scene)
    result_folder = os.path.join(results_folder, folder, scene)

    # Generate data
    for seed in data_seeds:
        _ = generate_dataset.run(scene, num_endmembers, spectral_size, image_size=image_size, output_dir=os.path.join(scene_folder, f'{scene}{seed}'), seed=seed)

    # Run experiment
    metrics = experiment.run(glob.glob(scene_folder + '/*'), num_endmembers, model_replicates=model_replicates, epochs=epochs, experiment_folder=result_folder, seed=experiment_seed, ae_loss=ae_loss)

    results[folder][scene] = metrics

clear_output()

for scene, metrics in results[folder].items():
    print("Results for scene: ", scene)
    display_results(metrics)

Results for scene:  chessboard


,SAD,PCC,MSE
index,,,
PCA,0.874 ± 0.139,0.387 ± 0.078,22.592 ± 8.969
N-FINDR + FCLS,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
VCA + FCLS,0.0 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
Dense AE,0.015 ± 0.065,0.009 ± 0.044,0.002 ± 0.008
Convolutional AE,0.042 ± 0.111,0.027 ± 0.074,0.002 ± 0.007
Transformer AE,0.002 ± 0.0,0.0 ± 0.0,0.0 ± 0.0
Convolutional Transformer AE,0.002 ± 0.0,0.0 ± 0.0,0.0 ± 0.0


Results for scene:  gaussian


,SAD,PCC,MSE
index,,,
PCA,1.051 ± 0.081,0.501 ± 0.068,5.387 ± 1.876
N-FINDR + FCLS,0.473 ± 0.042,0.179 ± 0.046,0.033 ± 0.001
VCA + FCLS,0.482 ± 0.047,0.182 ± 0.046,0.038 ± 0.009
Dense AE,0.255 ± 0.037,0.047 ± 0.02,0.01 ± 0.005
Convolutional AE,0.168 ± 0.033,0.021 ± 0.012,0.009 ± 0.006
Transformer AE,0.212 ± 0.043,0.033 ± 0.018,0.009 ± 0.005
Convolutional Transformer AE,0.215 ± 0.042,0.034 ± 0.019,0.009 ± 0.005


Results for scene:  dirichlet


,SAD,PCC,MSE
index,,,
PCA,0.847 ± 0.092,0.364 ± 0.049,3.811 ± 1.291
N-FINDR + FCLS,0.101 ± 0.04,0.008 ± 0.008,0.001 ± 0.0
VCA + FCLS,0.105 ± 0.031,0.009 ± 0.01,0.003 ± 0.003
Dense AE,0.003 ± 0.001,0.0 ± 0.0,0.012 ± 0.007
Convolutional AE,0.003 ± 0.001,0.0 ± 0.0,0.012 ± 0.006
Transformer AE,0.007 ± 0.002,0.0 ± 0.0,0.012 ± 0.006
Convolutional Transformer AE,0.017 ± 0.062,0.004 ± 0.02,0.013 ± 0.007


### +artefacts scenario

In [8]:
folder = 'artefacts'

results[folder] = {}
for scene in scenes:
    # Create folders
    scene_folder = os.path.join(data_folder, folder, scene)
    result_folder = os.path.join(results_folder, folder, scene)

    # Generate data
    for seed in data_seeds:
        _ = generate_dataset.run(scene, num_endmembers, spectral_size, image_size=image_size, output_dir=os.path.join(scene_folder, f'{scene}{seed}'), noise=True, baseline=True, cosmic_spikes=True, seed=seed)

    # Run experiment
    metrics = experiment.run(glob.glob(scene_folder + '/*'), num_endmembers + 1, model_replicates=model_replicates, epochs=epochs, experiment_folder=result_folder, seed=experiment_seed, ae_loss=ae_loss)

    results[folder][scene] = metrics

clear_output()

for scene, metrics in results[folder].items():
    print("Results for scene: ", scene)
    display_results(metrics)

Results for scene:  chessboard


,SAD,PCC,MSE
index,,,
PCA,0.819 ± 0.072,0.362 ± 0.065,79.38 ± 49.727
N-FINDR + FCLS,0.629 ± 0.111,0.226 ± 0.087,0.073 ± 0.036
VCA + FCLS,0.353 ± 0.159,0.115 ± 0.105,0.071 ± 0.042
Dense AE,0.072 ± 0.081,0.021 ± 0.044,0.042 ± 0.032
Convolutional AE,0.033 ± 0.003,0.001 ± 0.0,0.023 ± 0.013
Transformer AE,0.039 ± 0.031,0.002 ± 0.009,0.022 ± 0.011
Convolutional Transformer AE,0.033 ± 0.004,0.001 ± 0.0,0.021 ± 0.01


Results for scene:  gaussian


,SAD,PCC,MSE
index,,,
PCA,1.009 ± 0.082,0.489 ± 0.071,61.488 ± 48.586
N-FINDR + FCLS,0.828 ± 0.075,0.394 ± 0.076,0.041 ± 0.015
VCA + FCLS,0.609 ± 0.093,0.262 ± 0.085,0.059 ± 0.015
Dense AE,0.347 ± 0.137,0.097 ± 0.09,0.029 ± 0.021
Convolutional AE,0.385 ± 0.166,0.126 ± 0.104,0.027 ± 0.014
Transformer AE,0.393 ± 0.129,0.121 ± 0.084,0.026 ± 0.014
Convolutional Transformer AE,0.399 ± 0.164,0.134 ± 0.108,0.027 ± 0.015


Results for scene:  dirichlet


,SAD,PCC,MSE
index,,,
PCA,0.869 ± 0.069,0.385 ± 0.043,60.154 ± 48.33
N-FINDR + FCLS,0.599 ± 0.116,0.219 ± 0.109,0.033 ± 0.015
VCA + FCLS,0.392 ± 0.175,0.132 ± 0.108,0.036 ± 0.019
Dense AE,0.068 ± 0.07,0.009 ± 0.039,0.021 ± 0.011
Convolutional AE,0.068 ± 0.072,0.01 ± 0.041,0.017 ± 0.008
Transformer AE,0.073 ± 0.105,0.02 ± 0.066,0.017 ± 0.009
Convolutional Transformer AE,0.112 ± 0.14,0.041 ± 0.084,0.019 ± 0.01


### +realistic scenario

In [9]:
folder = 'realistic'

results[folder] = {}
for scene in scenes:
    # Create folders
    scene_folder = os.path.join(data_folder, folder, scene)
    result_folder = os.path.join(results_folder, folder, scene)

    # Generate data
    for seed in data_seeds:
        _ = generate_dataset.run(scene, num_endmembers, spectral_size, image_size=image_size, output_dir=os.path.join(scene_folder, f'{scene}{seed}'), noise=True, baseline=True, cosmic_spikes=True, realistic_endmembers=True, seed=seed)

    # Run experiment
    metrics = experiment.run(glob.glob(scene_folder + '/*'), num_endmembers + 1, model_replicates=model_replicates, epochs=epochs, experiment_folder=result_folder, seed=experiment_seed, ae_loss=ae_loss)

    results[folder][scene] = metrics

clear_output()

for scene, metrics in results[folder].items():
    print("Results for scene: ", scene)
    display_results(metrics)

Results for scene:  chessboard


,SAD,PCC,MSE
index,,,
PCA,0.967 ± 0.074,0.381 ± 0.093,88.132 ± 50.053
N-FINDR + FCLS,0.361 ± 0.062,0.19 ± 0.085,0.072 ± 0.04
VCA + FCLS,0.173 ± 0.079,0.091 ± 0.094,0.061 ± 0.038
Dense AE,0.045 ± 0.024,0.005 ± 0.01,0.03 ± 0.023
Convolutional AE,0.054 ± 0.013,0.003 ± 0.002,0.019 ± 0.008
Transformer AE,0.039 ± 0.005,0.001 ± 0.0,0.018 ± 0.009
Convolutional Transformer AE,0.04 ± 0.005,0.002 ± 0.001,0.018 ± 0.008


Results for scene:  gaussian


,SAD,PCC,MSE
index,,,
PCA,1.079 ± 0.045,0.491 ± 0.065,63.735 ± 48.513
N-FINDR + FCLS,0.478 ± 0.047,0.344 ± 0.065,0.043 ± 0.013
VCA + FCLS,0.4 ± 0.075,0.256 ± 0.093,0.063 ± 0.016
Dense AE,0.177 ± 0.05,0.05 ± 0.054,0.015 ± 0.011
Convolutional AE,0.168 ± 0.03,0.041 ± 0.02,0.011 ± 0.004
Transformer AE,0.156 ± 0.044,0.037 ± 0.028,0.011 ± 0.005
Convolutional Transformer AE,0.151 ± 0.033,0.034 ± 0.019,0.01 ± 0.004


Results for scene:  dirichlet


,SAD,PCC,MSE
index,,,
PCA,0.993 ± 0.089,0.403 ± 0.106,61.244 ± 48.733
N-FINDR + FCLS,0.299 ± 0.087,0.149 ± 0.098,0.03 ± 0.019
VCA + FCLS,0.229 ± 0.088,0.115 ± 0.098,0.03 ± 0.017
Dense AE,0.073 ± 0.012,0.004 ± 0.002,0.011 ± 0.006
Convolutional AE,0.071 ± 0.022,0.005 ± 0.007,0.008 ± 0.003
Transformer AE,0.085 ± 0.014,0.005 ± 0.002,0.008 ± 0.003
Convolutional Transformer AE,0.081 ± 0.012,0.005 ± 0.002,0.008 ± 0.003
